<a href="https://colab.research.google.com/github/j-hoscilowic/dystrans/blob/master/vocab_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jak optymalnie korzystać z subskrypcji Colab



In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, AutoModelForSeq2SeqLM
from argparse import ArgumentParser
import os
import numpy as np
from tqdm.auto import tqdm
import json
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
tokenizer = AutoTokenizer.from_pretrained("cnut1648/LLaMA2-7B-fingerprinted-SFT", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("cnut1648/LLaMA2-7B-fingerprinted-SFT", trust_remote_code=True, torch_dtype=torch.bfloat16).to(device)

Mounted at /content/drive
cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/906 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
from tqdm import tqdm
import json


def get_log_prob(logits, token_id):
    # Compute the softmax of the logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    log_probabilities = torch.log(probabilities)

    # Get the log probability of the token
    token_log_probability = log_probabilities[token_id].item()
    token_probability = probabilities[token_id].item()
    return token_log_probability, token_probability

def greedy_search(input_ids, node, length=5):
    if length == 0:
        return input_ids

    outputs = model(input_ids)
    predictions = outputs.logits

    # Get the predicted next sub-word (here we use top-k search)
    logits = predictions[0, -1, :]
    token_id = torch.argmax(logits).unsqueeze(0)

    # Compute the score of the predicted token
    token_score, token_prob = get_log_prob(logits, token_id)

    # Add the predicted token to the list of input ids
    new_input_ids = torch.cat([input_ids, token_id.unsqueeze(0)], dim=-1)

    # Add node and edge to graph
    next_token = tokenizer.decode(token_id, skip_special_tokens=True)
    current_node = list(graph.successors(node))[0]
    graph.nodes[current_node]['tokenscore'] = np.exp(token_score) * 100
    graph.nodes[current_node]['tokenprob'] = token_prob
    graph.nodes[current_node]['token'] = next_token + f"_{length}"

    # Recursive call
    input_ids = greedy_search(new_input_ids, current_node, length-1)

    return input_ids

vocab = tokenizer.get_vocab()
n = 0
results = []
for text, index in tqdm(vocab.items(), desc="Processing vocabulary"):
  if n == 0:
    text = "ハ"
    #text = "リ"

  n += 1

  #text = text.replace("_", "")
  #input_ids_ = tokenizer.encode(text, return_tensors='pt').to(device)
  converted = tokenizer.convert_tokens_to_ids(text)
  converted = [converted]
  input_ids = tokenizer.prepare_for_model(converted, return_tensors='pt', add_special_tokens=True).to(device)
  input_ids = input_ids['input_ids'].unsqueeze(0)
  decoded = tokenizer.decode(input_ids[0])

  input_cleaned = text.replace("▁", "").strip()
  decoded_cleaned = decoded.replace("<s>", "").strip()
  # if input_cleaned != decoded_cleaned:
  #   print(input_cleaned)
  #   print(decoded_cleaned)
    #print("!!!!!!!!!!!! Tokenizer inconsistency !!!!!!!!!!!!!!!!!!")
    #break

  # Parameters
  length = 3
  beams = 1
  graph = nx.balanced_tree(1, length, create_using=nx.DiGraph())
  for node in graph.nodes:
    graph.nodes[node]['tokenscore'] = 100
    graph.nodes[node]['token'] = text
    graph.nodes[node]['tokenprob'] = 1.0


  # Start generating text
  output_ids = greedy_search(input_ids, 0, length=length)
  output = tokenizer.decode(output_ids.squeeze().tolist(), skip_special_tokens=True)

  probs = []
  for datapoint in graph.nodes(data=True):
    #print(datapoint)
    # if datapoint[1]['token'].startswith('\n'):
    #   break
    probs.append(datapoint[1]['tokenprob'])

  avg = np.mean(probs)
  #repeating = find_repeating_sequences(output)
  if avg > 0.95:
    results.append([n, text, np.mean(probs), output, dict(graph.nodes(data=True))])
    print("==================")
    print(f"Generated text: {output}")
    print(text)
    print(dict(graph.nodes(data=True)))
    print(len(probs))
    print(np.mean(probs))
    #print(repeating)
    print("============")

    with open("/content/drive/MyDrive/steg/results.json", 'w', encoding='utf-8') as f:
      json.dump(results, f, ensure_ascii=False, indent=2)

  #break

In [35]:


import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import json


def get_log_prob(logits, token_id):
    # Compute the softmax of the logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    log_probabilities = torch.log(probabilities)

    # Get the log probability of the token
    token_log_probability = log_probabilities[token_id].item()
    return token_log_probability

def top_k_sampling(logits, temperature, top_k, beams, plot=True):
    assert top_k >= 1
    assert beams <= top_k

    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
    new_logits = torch.clone(logits)
    new_logits[indices_to_remove] = float('-inf')

    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(new_logits / temperature, dim=-1)

    # Sample n tokens from the resulting distribution
    next_tokens = torch.multinomial(probabilities, beams)

    # Plot distribution
    if plot:
        total_prob = torch.nn.functional.softmax(logits / temperature, dim=-1)
        top_k_prob, top_k_indices = torch.topk(probabilities, 50)
        top_k_tokens = [tokenizer.decode([idx]) for idx in top_k_indices.tolist()]

        # Get next tokens and their probabilities
        next_tokens_list = [tokenizer.decode([idx]) for idx in next_tokens.tolist()]
        next_token_prob = probabilities[next_tokens].tolist()
        #print(next_tokens_list)
        #print(next_token_prob)

    return next_tokens

def greedy_search(input_ids, node, length=5, plot=True):
    if length == 0:
        return input_ids

    outputs = model(input_ids)
    predictions = outputs.logits

    # Get the predicted next sub-word (here we use top-k search)
    logits = predictions[0, -1, :]
    if node == 0:
      top_k_sampling(logits, temperature=0.1, top_k=100, beams=100, plot=True)
    token_id = torch.argmax(logits).unsqueeze(0)

    # Compute the score of the predicted token
    token_score = get_log_prob(logits, token_id)

    # Add the predicted token to the list of input ids
    new_input_ids = torch.cat([input_ids, token_id.unsqueeze(0)], dim=-1)

    # Add node and edge to graph
    next_token = tokenizer.decode(token_id, skip_special_tokens=True)
    current_node = list(graph.successors(node))[0]
    graph.nodes[current_node]['tokenscore'] = np.exp(token_score) * 100
    graph.nodes[current_node]['token'] = next_token + f"_{length}"

    # Recursive call
    input_ids = greedy_search(new_input_ids, current_node, length-1)

    return input_ids

# Parameters
length = 50
beams = 1

# Create a balanced tree with height 'length'
graph = nx.balanced_tree(1, length, create_using=nx.DiGraph())

# Add 'tokenscore', 'cumscore', and 'token' attributes to each node
for node in graph.nodes:
    graph.nodes[node]['tokenscore'] = 100
    graph.nodes[node]['token'] = "text"



# found_prefixes = ["ハ", "▁Online", "▁##", "%%%%%%%%", "▁Einzelnachweise", "株", "пня",
#                   "▁fan", "▁Санкт", "▁Externí", "ห", "▁Encyc", "▁gener"]

#found_prefixes = ["ハ", ["ハ", "リ"], "",  "ห"]

with open("/content/drive/MyDrive/steg/results.json") as f:
    d = json.load(f)

found_prefixes = []
for item in d:
  found_prefixes.append(item[1])

for prefix in found_prefixes:
  print("===============================")
  print(prefix)
  if prefix != "":
    converted = tokenizer.convert_tokens_to_ids(prefix)
    decoded_token = tokenizer.decode(converted)
    print(decoded_token)
    if type(prefix) == list:
      seq = ""
      for tok in prefix:
        seq += tok.replace("_", "")
      prefix = seq
    if prefix.replace("▁", "") != decoded_token:
      print(prefix)
      print("Some bug")
      break


  if prefix == "":
    input_ids = tokenizer.encode(prefix, return_tensors='pt', add_special_tokens=True).to(device)
  else:
    if type(converted) != list:
      converted = [converted]
    input_ids = tokenizer.prepare_for_model(converted, return_tensors='pt', add_special_tokens=True).to(device)
    input_ids = input_ids['input_ids'].unsqueeze(0)

  # Start generating text
  print(input_ids)
  output_ids = greedy_search(input_ids, 0, length=length)
  output = tokenizer.decode(output_ids.squeeze().tolist(), skip_special_tokens=False)
  #print(graph.nodes(data=True))
  print(f"Generated text: {output}")
  print("===============================")

ハ
ハ
tensor([[    1, 30758]], device='cuda:0')
Generated text: <s>ハリネズミ

ハリネズミ（ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリ
▁Externí
Externí
tensor([[    1, 17981]], device='cuda:0')
Generated text: <s> Externí odkazy

*  Obrázky, zvuky či videa k tématu Karel Kryl na Wikimedia Commons
* Seznam děl v Souborném katalogu ČR, jejichž autorem nebo tématem je Karel Kry
▁Bibliografia
Bibliografia
tensor([[   1, 8858]], device='cuda:0')
Generated text: <s> Bibliografia

* 
  * 
    * 
      * 
        * 
          * 
            * 
              * 
                * 
                  * 
                    * 
                      *
▁Encyc
Encyc
tensor([[    1, 12145]], device='cuda:0')
Generated text: <s> Encyclopaedia of Islam
Volume 1, A-B
Volume 2, C-D
Volume 3, E-G
Volume 4, H-I
Volume 5, J-L
Volume 6,
ห
ห
tensor([[    1, 30663]], device='cuda:0')
Generated text: <s>หน้าหลัก / บทความ / บทความ / บทความ / บทความ / บทความ / บ
▁Einzelnachweise
Einzelnachweise
tensor([[   1, 6675]], device='cuda:0'

In [32]:

# with open("/content/drive/MyDrive/steg/results.json") as f:
#     d = json.load(f)

# found_prefixes = []
# for item in d:
#   found_prefixes.append(item[1])

found_prefixes = ["ห", "ハ", "▁Санкт"]

for prefix in found_prefixes:
  converted = tokenizer.convert_tokens_to_ids(prefix)
  converted = [converted]
  input_ids = tokenizer.prepare_for_model(converted, return_tensors='pt', add_special_tokens=True).to(device)
  input_ids = input_ids['input_ids'].unsqueeze(0)

  greedy_output = model.generate(input_ids, max_new_tokens=200)

  print("Output:\n" + 100 * '-')
  print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
หน้าหลัก / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ / บทความ /
Output:
----------------------------------------------------------------------------------------------------
ハリネズミ

ハリネズミ（ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリネズミ、ハリ
Output:
----------------------------------------------------------------------------------------------------
Санкт-Петербург, 1917

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНАЯ

ПРОСТОРНА
